[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/ml-math-with-densworld/blob/main/modules/03-calculus/notebooks/04-chain-rule-backprop.ipynb)

# Lesson 4: The Chain Rule and Backpropagation

*"The Tower of Mirado does not respond directly to my actions. My stratagems affect my troops' morale, which affects their effectiveness, which affects the damage we inflict, which affects the Tower's defenses, which affects my progress. Layer upon layer of cause and effect. To understand how my initial choice propagates through this chain—this is the deepest question of optimization."*  
— The Colonel, Year 20 of the Siege

---

## The Core Insight

Neural networks are **nested functions**—layers upon layers of transformations. To train them, we need the gradient of the loss with respect to every weight in every layer.

The **chain rule** tells us how to compute these gradients by **propagating sensitivity backward** through the network.

If $y = f(g(x))$, then:

$$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$$

In words: *The sensitivity of y to x is the product of sensitivities along the chain.*

---

## Learning Objectives

By the end of this lesson, you will:
1. Apply the chain rule to composite functions
2. Understand computational graphs and forward/backward passes
3. Implement backpropagation from scratch for a simple network
4. Connect backpropagation to the Colonel's layered decision-making

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Nice plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Colab-ready data loading
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/ml-math-with-densworld/main/data/"

# Load data
stratagem = pd.read_csv(BASE_URL + "stratagem_details.csv")
expedition = pd.read_csv(BASE_URL + "expedition_outcomes.csv")

print(f"Loaded {len(stratagem)} stratagem records")
print(f"Loaded {len(expedition)} expedition records")

## Part 1: The Chain Rule — Sensitivity Through Layers

### The Simple Case: Two Functions

Consider:
- $g(x) = x^2$ (inner function)
- $f(g) = \sin(g)$ (outer function)
- $y = f(g(x)) = \sin(x^2)$ (composite)

How does $y$ change when we nudge $x$?

$$\frac{dy}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx} = \cos(x^2) \cdot 2x$$

In [ ]:
# Demonstrate chain rule
def g(x):
    return x**2

def f(u):
    return np.sin(u)

def composite(x):
    return f(g(x))

def dg_dx(x):
    return 2 * x

def df_dg(u):
    return np.cos(u)

def dy_dx_chain(x):
    """Chain rule: dy/dx = df/dg * dg/dx"""
    return df_dg(g(x)) * dg_dx(x)

# Numerical verification
def numerical_derivative(func, x, h=1e-5):
    return (func(x + h) - func(x - h)) / (2 * h)

# Test at several points
test_points = [0.5, 1.0, 1.5, 2.0]

print("Chain Rule Verification:")
print("=" * 60)
print(f"{'x':>8} | {'Chain Rule':>15} | {'Numerical':>15} | Match?")
print("-" * 60)

for x in test_points:
    chain = dy_dx_chain(x)
    numerical = numerical_derivative(composite, x)
    match = "Yes" if np.isclose(chain, numerical) else "No"
    print(f"{x:>8.2f} | {chain:>15.6f} | {numerical:>15.6f} | {match}")

In [ ]:
# Visualize the chain rule
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

x = np.linspace(0.1, 2.5, 100)

# Top left: The composite function
axes[0, 0].plot(x, composite(x), 'b-', linewidth=2)
axes[0, 0].set_xlabel('x', fontsize=11)
axes[0, 0].set_ylabel('y = sin(x²)', fontsize=11)
axes[0, 0].set_title('Composite Function: f(g(x)) = sin(x²)', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

# Top right: Inner function g(x) = x²
axes[0, 1].plot(x, g(x), 'g-', linewidth=2, label='g(x) = x²')
axes[0, 1].plot(x, dg_dx(x), 'g--', linewidth=2, label="g'(x) = 2x")
axes[0, 1].set_xlabel('x', fontsize=11)
axes[0, 1].set_ylabel('g(x) and g\'(x)', fontsize=11)
axes[0, 1].set_title('Inner Function and Its Derivative', fontsize=12)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Outer function f(u) = sin(u)
u = np.linspace(0, 6, 100)
axes[1, 0].plot(u, f(u), 'r-', linewidth=2, label='f(u) = sin(u)')
axes[1, 0].plot(u, df_dg(u), 'r--', linewidth=2, label="f'(u) = cos(u)")
axes[1, 0].set_xlabel('u = g(x)', fontsize=11)
axes[1, 0].set_ylabel('f(u) and f\'(u)', fontsize=11)
axes[1, 0].set_title('Outer Function and Its Derivative', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: The chain rule derivative
axes[1, 1].plot(x, dy_dx_chain(x), 'purple', linewidth=2, label="dy/dx = cos(x²) · 2x")
axes[1, 1].axhline(0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('x', fontsize=11)
axes[1, 1].set_ylabel('dy/dx', fontsize=11)
axes[1, 1].set_title('Chain Rule Result: dy/dx = df/dg · dg/dx', fontsize=12)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: The Colonel's Chain of Causation

*"When I commit more personnel, it does not directly reduce the Tower's defenses. Instead, it flows through channels: more personnel → higher morale → more effective assaults → more damage → reduced defenses → progress. Each link in the chain multiplies or diminishes the effect."*  
— The Colonel

Let's model this chain:

1. **Personnel** → affects **Morale**
2. **Morale** → affects **Assault Effectiveness**
3. **Assault Effectiveness** → affects **Progress**

This is exactly like a neural network with three layers!

In [ ]:
# Model the Colonel's chain of causation
def personnel_to_morale(personnel, w1=0.001):
    """More personnel → higher morale (with saturation)"""
    return np.tanh(w1 * personnel)

def morale_to_effectiveness(morale, w2=1.5):
    """Higher morale → more effective assaults"""
    return w2 * morale

def effectiveness_to_progress(effectiveness, w3=0.05):
    """Higher effectiveness → more progress (with noise)"""
    return w3 * effectiveness

def full_chain(personnel, w1=0.001, w2=1.5, w3=0.05):
    """Full forward pass through the chain."""
    morale = personnel_to_morale(personnel, w1)
    effectiveness = morale_to_effectiveness(morale, w2)
    progress = effectiveness_to_progress(effectiveness, w3)
    return progress

# Compute chain rule derivative: d(progress)/d(personnel)
def chain_rule_derivative(personnel, w1=0.001, w2=1.5, w3=0.05):
    """Apply chain rule: d(progress)/d(personnel) = d(progress)/d(eff) * d(eff)/d(morale) * d(morale)/d(personnel)"""
    morale = personnel_to_morale(personnel, w1)
    
    # d(morale)/d(personnel) = w1 * sech²(w1 * personnel) = w1 * (1 - tanh²(w1 * personnel))
    d_morale_d_personnel = w1 * (1 - morale**2)
    
    # d(effectiveness)/d(morale) = w2
    d_eff_d_morale = w2
    
    # d(progress)/d(effectiveness) = w3
    d_progress_d_eff = w3
    
    # Chain rule: multiply all the derivatives
    d_progress_d_personnel = d_progress_d_eff * d_eff_d_morale * d_morale_d_personnel
    
    return d_progress_d_personnel

# Visualize
personnel_range = np.linspace(0, 100, 100)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Chain components
morale = [personnel_to_morale(p) for p in personnel_range]
effectiveness = [morale_to_effectiveness(m) for m in morale]
progress = [effectiveness_to_progress(e) for e in effectiveness]

axes[0].plot(personnel_range, morale, 'b-', linewidth=2)
axes[0].set_xlabel('Personnel', fontsize=11)
axes[0].set_ylabel('Morale', fontsize=11)
axes[0].set_title('Layer 1: Personnel → Morale', fontsize=12)

axes[1].plot(morale, effectiveness, 'g-', linewidth=2)
axes[1].set_xlabel('Morale', fontsize=11)
axes[1].set_ylabel('Effectiveness', fontsize=11)
axes[1].set_title('Layer 2: Morale → Effectiveness', fontsize=12)

axes[2].plot(effectiveness, progress, 'r-', linewidth=2)
axes[2].set_xlabel('Effectiveness', fontsize=11)
axes[2].set_ylabel('Progress', fontsize=11)
axes[2].set_title('Layer 3: Effectiveness → Progress', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Show end-to-end and the chain rule derivative
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: End-to-end function
progress_values = [full_chain(p) for p in personnel_range]
axes[0].plot(personnel_range, progress_values, 'purple', linewidth=2)
axes[0].set_xlabel('Personnel', fontsize=11)
axes[0].set_ylabel('Progress', fontsize=11)
axes[0].set_title('End-to-End: Personnel → Progress', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Right: Chain rule derivative
chain_derivs = [chain_rule_derivative(p) for p in personnel_range]
numerical_derivs = [numerical_derivative(full_chain, p) for p in personnel_range]

axes[1].plot(personnel_range, chain_derivs, 'r-', linewidth=2, label='Chain Rule')
axes[1].plot(personnel_range, numerical_derivs, 'b--', linewidth=2, label='Numerical')
axes[1].set_xlabel('Personnel', fontsize=11)
axes[1].set_ylabel('d(Progress)/d(Personnel)', fontsize=11)
axes[1].set_title('Gradient Through the Chain\n(Chain Rule = Numerical)', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The chain rule derivative matches the numerical derivative exactly!")
print("This is how backpropagation computes gradients efficiently.")

## Part 3: Computational Graphs

Neural networks can be represented as **computational graphs**:
- **Nodes** are operations (add, multiply, activation functions)
- **Edges** carry values forward and gradients backward

### The Two Passes:

1. **Forward Pass**: Compute output from inputs
2. **Backward Pass**: Compute gradients from output to inputs

Let's implement a simple computational graph:

In [ ]:
class ComputeNode:
    """
    A node in a computational graph.
    Tracks value, gradient, and operation.
    """
    def __init__(self, value, name=""):
        self.value = value
        self.grad = 0.0  # Gradient will be accumulated here
        self.name = name
        self.children = []  # Operations that produced this node
    
    def __repr__(self):
        return f"{self.name}(value={self.value:.4f}, grad={self.grad:.4f})"

def multiply(a, b, name=""):
    """Multiply two nodes."""
    result = ComputeNode(a.value * b.value, name=name)
    
    def backward():
        # d(a*b)/da = b, d(a*b)/db = a
        a.grad += result.grad * b.value
        b.grad += result.grad * a.value
    
    result.children = [(a, b, backward)]
    return result

def add(a, b, name=""):
    """Add two nodes."""
    result = ComputeNode(a.value + b.value, name=name)
    
    def backward():
        # d(a+b)/da = 1, d(a+b)/db = 1
        a.grad += result.grad * 1
        b.grad += result.grad * 1
    
    result.children = [(a, b, backward)]
    return result

def sigmoid(a, name=""):
    """Sigmoid activation."""
    sig_val = 1 / (1 + np.exp(-a.value))
    result = ComputeNode(sig_val, name=name)
    
    def backward():
        # d(sigmoid)/da = sigmoid * (1 - sigmoid)
        a.grad += result.grad * sig_val * (1 - sig_val)
    
    result.children = [(a, None, backward)]
    return result

def backward_pass(output):
    """
    Perform backward pass from output node.
    Uses topological sort to ensure correct order.
    """
    # Build topological order
    topo = []
    visited = set()
    
    def build_topo(node):
        if id(node) not in visited:
            visited.add(id(node))
            for child_tuple in node.children:
                for child in child_tuple[:-1]:  # Exclude the backward function
                    if child is not None:
                        build_topo(child)
            topo.append(node)
    
    build_topo(output)
    
    # Backward pass
    output.grad = 1.0  # Start with gradient of 1 at output
    for node in reversed(topo):
        for child_tuple in node.children:
            backward_fn = child_tuple[-1]
            backward_fn()

# Example: Simple computation
print("Example: y = (x * w1 + b1) * w2")
print("=" * 50)

# Forward pass
x = ComputeNode(2.0, name="x")
w1 = ComputeNode(3.0, name="w1")
b1 = ComputeNode(1.0, name="b1")
w2 = ComputeNode(0.5, name="w2")

# Compute graph
xw1 = multiply(x, w1, name="x*w1")
xw1_b1 = add(xw1, b1, name="x*w1+b1")
y = multiply(xw1_b1, w2, name="y")

print(f"Forward pass:")
print(f"  x = {x.value}, w1 = {w1.value}, b1 = {b1.value}, w2 = {w2.value}")
print(f"  x * w1 = {xw1.value}")
print(f"  x * w1 + b1 = {xw1_b1.value}")
print(f"  y = (x * w1 + b1) * w2 = {y.value}")

# Backward pass
backward_pass(y)

print(f"\nBackward pass (gradients):")
print(f"  dy/dx = {x.grad}")
print(f"  dy/dw1 = {w1.grad}")
print(f"  dy/db1 = {b1.grad}")
print(f"  dy/dw2 = {w2.grad}")

In [ ]:
# Verify with numerical derivatives
def compute_y(x_val, w1_val, b1_val, w2_val):
    return (x_val * w1_val + b1_val) * w2_val

h = 1e-5

print("Verification with Numerical Derivatives:")
print("=" * 50)

# dy/dx
dy_dx_numerical = (compute_y(2.0 + h, 3.0, 1.0, 0.5) - compute_y(2.0 - h, 3.0, 1.0, 0.5)) / (2*h)
print(f"dy/dx: Backprop = {x.grad:.4f}, Numerical = {dy_dx_numerical:.4f}")

# dy/dw1
dy_dw1_numerical = (compute_y(2.0, 3.0 + h, 1.0, 0.5) - compute_y(2.0, 3.0 - h, 1.0, 0.5)) / (2*h)
print(f"dy/dw1: Backprop = {w1.grad:.4f}, Numerical = {dy_dw1_numerical:.4f}")

# dy/db1
dy_db1_numerical = (compute_y(2.0, 3.0, 1.0 + h, 0.5) - compute_y(2.0, 3.0, 1.0 - h, 0.5)) / (2*h)
print(f"dy/db1: Backprop = {b1.grad:.4f}, Numerical = {dy_db1_numerical:.4f}")

# dy/dw2
dy_dw2_numerical = (compute_y(2.0, 3.0, 1.0, 0.5 + h) - compute_y(2.0, 3.0, 1.0, 0.5 - h)) / (2*h)
print(f"dy/dw2: Backprop = {w2.grad:.4f}, Numerical = {dy_dw2_numerical:.4f}")

## Part 4: A Simple Neural Network from Scratch

Now let's implement a complete neural network with backpropagation:

- **Input layer**: Takes features
- **Hidden layer**: Linear transformation + activation
- **Output layer**: Final prediction
- **Loss**: Mean squared error

In [ ]:
class SimpleNeuralNetwork:
    """
    A simple 2-layer neural network with backpropagation.
    Architecture: Input -> Hidden (with ReLU) -> Output
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights with small random values
        self.W1 = np.random.randn(input_size, hidden_size) * 0.1
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.1
        self.b2 = np.zeros((1, output_size))
        
        # Cache for backward pass
        self.cache = {}
    
    def relu(self, x):
        """ReLU activation."""
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        """Derivative of ReLU."""
        return (x > 0).astype(float)
    
    def forward(self, X):
        """
        Forward pass.
        
        Returns predictions and caches intermediate values.
        """
        # Layer 1
        self.cache['X'] = X
        self.cache['Z1'] = X @ self.W1 + self.b1
        self.cache['A1'] = self.relu(self.cache['Z1'])
        
        # Layer 2 (output)
        self.cache['Z2'] = self.cache['A1'] @ self.W2 + self.b2
        
        return self.cache['Z2']  # No activation on output for regression
    
    def backward(self, y_true, y_pred):
        """
        Backward pass using the chain rule.
        
        Computes gradients for all weights and biases.
        """
        m = y_true.shape[0]  # Number of samples
        
        # Output layer gradients
        # dL/dZ2 = dL/dy_pred * dy_pred/dZ2 = 2*(y_pred - y_true) / m * 1
        dZ2 = 2 * (y_pred - y_true) / m
        
        # dL/dW2 = dL/dZ2 * dZ2/dW2 = A1.T @ dZ2
        self.dW2 = self.cache['A1'].T @ dZ2
        self.db2 = np.sum(dZ2, axis=0, keepdims=True)
        
        # Hidden layer gradients (chain rule continues)
        # dL/dA1 = dL/dZ2 * dZ2/dA1 = dZ2 @ W2.T
        dA1 = dZ2 @ self.W2.T
        
        # dL/dZ1 = dL/dA1 * dA1/dZ1 = dA1 * relu'(Z1)
        dZ1 = dA1 * self.relu_derivative(self.cache['Z1'])
        
        # dL/dW1 = dL/dZ1 * dZ1/dW1 = X.T @ dZ1
        self.dW1 = self.cache['X'].T @ dZ1
        self.db1 = np.sum(dZ1, axis=0, keepdims=True)
    
    def update(self, learning_rate):
        """Update weights using gradients."""
        self.W2 -= learning_rate * self.dW2
        self.b2 -= learning_rate * self.db2
        self.W1 -= learning_rate * self.dW1
        self.b1 -= learning_rate * self.db1
    
    def loss(self, y_true, y_pred):
        """Mean squared error loss."""
        return np.mean((y_pred - y_true)**2)

print("SimpleNeuralNetwork class defined!")
print("Architecture: Input -> Hidden (ReLU) -> Output")

In [ ]:
# Prepare data from stratagem records
features = ['personnel_committed', 'supply_cost', 'risk_level', 'colonel_confidence']

X = stratagem[features].values
y = stratagem['progress_delta'].values.reshape(-1, 1)

# Normalize
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std

y_mean, y_std = y.mean(), y.std()
y_norm = (y - y_mean) / y_std

# Train/test split
split = int(0.8 * len(X))
X_train, X_test = X_norm[:split], X_norm[split:]
y_train, y_test = y_norm[:split], y_norm[split:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Train the neural network
nn = SimpleNeuralNetwork(input_size=4, hidden_size=8, output_size=1)

learning_rate = 0.01
num_epochs = 500

train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Forward pass
    y_pred_train = nn.forward(X_train)
    
    # Compute loss
    train_loss = nn.loss(y_train, y_pred_train)
    train_losses.append(train_loss)
    
    # Backward pass (this is backpropagation!)
    nn.backward(y_train, y_pred_train)
    
    # Update weights
    nn.update(learning_rate)
    
    # Compute test loss
    y_pred_test = nn.forward(X_test)
    test_loss = nn.loss(y_test, y_pred_test)
    test_losses.append(test_loss)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}: Train Loss = {train_loss:.6f}, Test Loss = {test_loss:.6f}")

print(f"\nFinal: Train Loss = {train_losses[-1]:.6f}, Test Loss = {test_losses[-1]:.6f}")

In [ ]:
# Visualize training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Loss curves
axes[0].plot(train_losses, 'b-', linewidth=2, label='Train')
axes[0].plot(test_losses, 'r-', linewidth=2, label='Test')
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('MSE Loss', fontsize=11)
axes[0].set_title('Training Progress (Backpropagation in Action)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: Predictions vs actual
y_pred_final = nn.forward(X_test)
axes[1].scatter(y_test, y_pred_final, alpha=0.6, s=40)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect prediction')
axes[1].set_xlabel('Actual (normalized)', fontsize=11)
axes[1].set_ylabel('Predicted (normalized)', fontsize=11)
axes[1].set_title('Predictions vs Actual on Test Set', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

## Part 5: Visualizing Gradients Through Layers

Let's trace how gradients flow backward through our network:

In [ ]:
# Analyze gradient magnitudes through layers
# Run one forward and backward pass to get fresh gradients
y_pred = nn.forward(X_train)
nn.backward(y_train, y_pred)

# Gradient statistics
print("Gradient Statistics Through Layers:")
print("=" * 60)

layers = [
    ('Layer 1 Weights (dW1)', nn.dW1),
    ('Layer 1 Biases (db1)', nn.db1),
    ('Layer 2 Weights (dW2)', nn.dW2),
    ('Layer 2 Biases (db2)', nn.db2)
]

for name, grad in layers:
    print(f"{name:30}: mean={grad.mean():.6f}, std={grad.std():.6f}, "
          f"min={grad.min():.6f}, max={grad.max():.6f}")

# Visualize gradient distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(nn.dW1.flatten(), bins=30, alpha=0.7, label='Layer 1 Weights', color='blue')
axes[0].hist(nn.dW2.flatten(), bins=30, alpha=0.7, label='Layer 2 Weights', color='red')
axes[0].set_xlabel('Gradient Value', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Gradient Distributions', fontsize=12)
axes[0].legend()

# Gradient magnitude over training
# Re-run training to collect gradient norms
nn2 = SimpleNeuralNetwork(input_size=4, hidden_size=8, output_size=1)
grad_norms_W1 = []
grad_norms_W2 = []

for epoch in range(200):
    y_pred = nn2.forward(X_train)
    nn2.backward(y_train, y_pred)
    
    grad_norms_W1.append(np.linalg.norm(nn2.dW1))
    grad_norms_W2.append(np.linalg.norm(nn2.dW2))
    
    nn2.update(0.01)

axes[1].plot(grad_norms_W1, 'b-', linewidth=2, label='Layer 1 (dW1)')
axes[1].plot(grad_norms_W2, 'r-', linewidth=2, label='Layer 2 (dW2)')
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Gradient Norm', fontsize=11)
axes[1].set_title('Gradient Magnitude Over Training', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

## Part 6: The Vanishing/Exploding Gradient Problem

In deep networks, gradients can become very small (vanishing) or very large (exploding) as they flow backward through many layers.

*"The deeper the chain of causation, the fainter the signal becomes. By the time I trace the effect of a decision made years ago through all the intervening events, the connection is nearly invisible. This is the vanishing gradient—the forgetting of distant causes."*  
— The Colonel

In [ ]:
# Demonstrate vanishing gradients with sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Chain of sigmoid layers
def deep_chain_gradient(x, num_layers):
    """Compute gradient through a chain of sigmoid layers."""
    # Forward pass
    activations = [x]
    current = x
    for _ in range(num_layers):
        current = sigmoid(current)
        activations.append(current)
    
    # Backward pass (chain rule)
    gradient = 1.0
    for i in range(num_layers - 1, -1, -1):
        gradient *= sigmoid_derivative(activations[i])
    
    return gradient

# Test with different depths
depths = [1, 5, 10, 20, 50]
x_values = np.linspace(-2, 2, 100)

fig, ax = plt.subplots(figsize=(10, 6))

for depth in depths:
    gradients = [deep_chain_gradient(x, depth) for x in x_values]
    ax.plot(x_values, gradients, linewidth=2, label=f'Depth = {depth}')

ax.set_xlabel('Input x', fontsize=11)
ax.set_ylabel('Gradient Magnitude', fontsize=11)
ax.set_title('Vanishing Gradients in Deep Sigmoid Networks', fontsize=12)
ax.legend()
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("As depth increases, gradients vanish exponentially!")
print("This is why ReLU and skip connections were invented.")

---

## Exercises

### Exercise 1: Manual Backpropagation

For the function $f(x) = (x^2 + 3)^3$, use the chain rule to compute $\frac{df}{dx}$ and verify numerically.

In [ ]:
# Exercise 1: Chain rule by hand

def f(x):
    return (x**2 + 3)**3

def df_dx_chain_rule(x):
    """
    Chain rule:
    Let u = x^2 + 3
    f = u^3
    
    df/dx = df/du * du/dx
          = 3u^2 * 2x
          = 3(x^2 + 3)^2 * 2x
          = 6x(x^2 + 3)^2
    """
    return 6 * x * (x**2 + 3)**2

# Verify
print("Exercise 1: Chain Rule Verification")
print("=" * 50)

for x in [1, 2, 3]:
    analytical = df_dx_chain_rule(x)
    numerical = numerical_derivative(f, x)
    print(f"x = {x}: Chain Rule = {analytical:.4f}, Numerical = {numerical:.4f}")

### Exercise 2: Add a Third Layer

Extend the SimpleNeuralNetwork class to have two hidden layers instead of one.

In [ ]:
# Exercise 2: Three-layer network

class ThreeLayerNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))
        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))
        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))
        self.cache = {}
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    def forward(self, X):
        self.cache['X'] = X
        self.cache['Z1'] = X @ self.W1 + self.b1
        self.cache['A1'] = self.relu(self.cache['Z1'])
        self.cache['Z2'] = self.cache['A1'] @ self.W2 + self.b2
        self.cache['A2'] = self.relu(self.cache['Z2'])
        self.cache['Z3'] = self.cache['A2'] @ self.W3 + self.b3
        return self.cache['Z3']
    
    def backward(self, y_true, y_pred):
        m = y_true.shape[0]
        
        # Output layer
        dZ3 = 2 * (y_pred - y_true) / m
        self.dW3 = self.cache['A2'].T @ dZ3
        self.db3 = np.sum(dZ3, axis=0, keepdims=True)
        
        # Hidden layer 2
        dA2 = dZ3 @ self.W3.T
        dZ2 = dA2 * self.relu_derivative(self.cache['Z2'])
        self.dW2 = self.cache['A1'].T @ dZ2
        self.db2 = np.sum(dZ2, axis=0, keepdims=True)
        
        # Hidden layer 1
        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * self.relu_derivative(self.cache['Z1'])
        self.dW1 = self.cache['X'].T @ dZ1
        self.db1 = np.sum(dZ1, axis=0, keepdims=True)
    
    def update(self, lr):
        self.W3 -= lr * self.dW3
        self.b3 -= lr * self.db3
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1
    
    def loss(self, y_true, y_pred):
        return np.mean((y_pred - y_true)**2)

# Train
nn3 = ThreeLayerNetwork(4, 8, 4, 1)
losses_3layer = []

for epoch in range(300):
    y_pred = nn3.forward(X_train)
    loss = nn3.loss(y_train, y_pred)
    losses_3layer.append(loss)
    nn3.backward(y_train, y_pred)
    nn3.update(0.01)

print(f"Three-layer network: Final loss = {losses_3layer[-1]:.6f}")

# Compare to 2-layer
plt.figure(figsize=(10, 5))
plt.plot(train_losses[:300], 'b-', label='2-layer network')
plt.plot(losses_3layer, 'r-', label='3-layer network')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Comparing 2-layer vs 3-layer Networks')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Exercise 3: Gradient Checking

Implement a gradient check that verifies backpropagation is correct by comparing to numerical gradients.

In [ ]:
# Exercise 3: Gradient checking

def gradient_check(nn, X, y, epsilon=1e-5):
    """
    Check gradients by comparing to numerical approximation.
    """
    # Forward and backward pass
    y_pred = nn.forward(X)
    nn.backward(y, y_pred)
    
    # Check W1 gradients
    numerical_dW1 = np.zeros_like(nn.W1)
    
    for i in range(nn.W1.shape[0]):
        for j in range(nn.W1.shape[1]):
            # Plus epsilon
            nn.W1[i, j] += epsilon
            y_plus = nn.forward(X)
            loss_plus = nn.loss(y, y_plus)
            
            # Minus epsilon
            nn.W1[i, j] -= 2 * epsilon
            y_minus = nn.forward(X)
            loss_minus = nn.loss(y, y_minus)
            
            # Restore
            nn.W1[i, j] += epsilon
            
            # Numerical gradient
            numerical_dW1[i, j] = (loss_plus - loss_minus) / (2 * epsilon)
    
    # Compare
    difference = np.linalg.norm(numerical_dW1 - nn.dW1) / (np.linalg.norm(numerical_dW1) + np.linalg.norm(nn.dW1))
    
    return difference, numerical_dW1, nn.dW1

# Test
nn_check = SimpleNeuralNetwork(4, 4, 1)
X_small = X_train[:10]
y_small = y_train[:10]

diff, num_grad, bp_grad = gradient_check(nn_check, X_small, y_small)

print("Gradient Check Results:")
print("=" * 50)
print(f"Relative difference: {diff:.10f}")
print(f"(Should be < 1e-5 for correct implementation)")

if diff < 1e-5:
    print("\nGradient check PASSED!")
else:
    print("\nGradient check FAILED. Check backprop implementation.")

### Exercise 4: The Colonel's Decision Tree

Model a multi-stage decision process where each stage feeds into the next, and compute how the final outcome's gradient flows back to the initial decision.

In [ ]:
# Exercise 4: The Colonel's decision chain

class ColonelDecisionChain:
    """
    Model the Colonel's multi-stage decision process:
    
    Initial Decision → Troop Morale → Assault Success → Progress
    """
    
    def __init__(self):
        # Weights for each stage
        self.w_decision_to_morale = 0.5
        self.w_morale_to_assault = 0.8
        self.w_assault_to_progress = 0.3
    
    def forward(self, decision):
        """Forward pass through the decision chain."""
        self.decision = decision
        self.morale = np.tanh(self.w_decision_to_morale * decision)
        self.assault = self.w_morale_to_assault * self.morale
        self.progress = self.w_assault_to_progress * self.assault
        return self.progress
    
    def backward(self):
        """Backward pass: compute d(progress)/d(decision)."""
        # d(progress)/d(assault)
        d_progress_d_assault = self.w_assault_to_progress
        
        # d(assault)/d(morale)
        d_assault_d_morale = self.w_morale_to_assault
        
        # d(morale)/d(decision) = w * (1 - tanh^2(w * decision))
        d_morale_d_decision = self.w_decision_to_morale * (1 - self.morale**2)
        
        # Chain rule
        self.gradient = d_progress_d_assault * d_assault_d_morale * d_morale_d_decision
        return self.gradient

# Test
chain = ColonelDecisionChain()

decisions = np.linspace(-5, 5, 100)
progress_values = [chain.forward(d) for d in decisions]
gradients = [chain.backward() for d in decisions for _ in [chain.forward(d)]]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(decisions, progress_values, 'b-', linewidth=2)
axes[0].set_xlabel('Initial Decision', fontsize=11)
axes[0].set_ylabel('Final Progress', fontsize=11)
axes[0].set_title('Colonel\'s Decision Chain: Input → Output', fontsize=12)
axes[0].grid(True, alpha=0.3)

gradients_clean = []
for d in decisions:
    chain.forward(d)
    gradients_clean.append(chain.backward())

axes[1].plot(decisions, gradients_clean, 'r-', linewidth=2)
axes[1].set_xlabel('Initial Decision', fontsize=11)
axes[1].set_ylabel('d(Progress)/d(Decision)', fontsize=11)
axes[1].set_title('Gradient Through the Chain (Backpropagation)', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The gradient tells the Colonel: 'If you increase your initial decision,")
print("here is how much your final progress will change.'")

---

## Summary

| Concept | Key Insight | Colonel's Siege Example |
|---------|-------------|------------------------|
| **Chain Rule** | Multiply sensitivities along the chain | d(progress)/d(decision) = d(progress)/d(assault) × d(assault)/d(morale) × d(morale)/d(decision) |
| **Computational Graph** | Break computation into nodes and edges | Each stage of the siege is a node |
| **Forward Pass** | Compute output from inputs | Decision → Morale → Assault → Progress |
| **Backward Pass** | Compute gradients from output to inputs | Propagate sensitivity backward through the chain |
| **Backpropagation** | Efficient application of chain rule | One forward + one backward pass gives all gradients |
| **Vanishing Gradients** | Deep chains attenuate gradients | Effects of long-ago decisions become invisible |

---

## Key Takeaways

1. **The chain rule is the foundation**: It tells us how to propagate sensitivity through nested functions.

2. **Backpropagation is efficient**: One forward pass, one backward pass, and we have all gradients.

3. **Computational graphs make it systematic**: Any differentiable computation can be broken into nodes with local gradients.

4. **Depth can cause problems**: Vanishing/exploding gradients are real challenges in deep networks.

5. **This is how neural networks learn**: Every adjustment to every weight in every layer comes from backpropagation.

---

## Module 3 Complete!

You have now completed the Calculus module. You understand:

1. **Derivatives as sensitivity** — how outputs respond to input changes
2. **Gradients as compasses** — vectors pointing uphill in parameter space
3. **Gradient descent** — the core optimization algorithm
4. **Backpropagation** — efficient computation of gradients in deep networks

In **Module 4: Applied ML**, we'll bring everything together: statistics, linear algebra, and calculus—to derive linear regression, understand regularization, and build a complete machine learning pipeline.

*"Twenty years of siegecraft have taught me the gradient. Twenty years of small adjustments, each informed by the last, each feeding into the next. The Tower has not yet fallen—but I understand now the shape of the landscape I navigate. This is the power of calculus: not to see the future, but to feel which way is down."*  
— The Colonel, final entry, Year 20